In [64]:
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing import sequence
import numpy as np

In [65]:
import json

In [66]:
with open('data/idx2vocab.json',encoding="utf-8") as file_obj:
    idx2vocab=json.load(file_obj)
    
with open('data/vocab2idx.json',encoding="utf-8") as file_obj:
    vocab2idx=json.load(file_obj)
    
with open('data/idx2label.json',encoding="utf-8") as file_obj:
    idx2label=json.load(file_obj)
    
with open('data/label2idx.json',encoding="utf-8") as file_obj:
    label2idx=json.load(file_obj)

In [67]:
model = models.load_model("model/output/bilstm_crf_ner", compile=False)

In [68]:
#提取转移矩阵参数
trans_params = model.get_layer('crf').get_weights()[0]
# 获得BiLSTM的输出logits
sub_model = models.Model(inputs=model.get_layer('input_ids').input,
                        outputs=model.get_layer('dense').output)

def predict(model, inputs, input_lens):
    logits = sub_model.predict(inputs)
    # 获取CRF层的转移矩阵
    # crf_decode：viterbi解码获得结果
    pred_seq, viterbi_score = tfa.text.crf_decode(logits, trans_params, input_lens)
    return pred_seq

In [69]:
def pre_data(sentence,vocab2idx,idx2label):
    maxlen=100
    sent_chars = list(sentence)
    sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['UNK'] for word in sent_chars]
    sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
    test_lens = np.array([100])

    pred_seq = predict(model, sent2id_new, test_lens)
    y_label = pred_seq.numpy().reshape(1, -1)[0]
    y_ner = [idx2label[str(i)] for i in y_label][-len(sent_chars):]
    return y_ner

In [71]:
pre_data('月经不调吃哪些药物',vocab2idx,idx2label)

['B-dis', 'I-dis', 'I-dis', 'E-dis', 'O', 'O', 'O', 'O', 'O']